In [1]:
import pandas as pd
from sklearn.model_selection import ParameterGrid
from neuralprophet import NeuralProphet, set_log_level, set_random_seed
from features import add_stock_price_feature

# Disable logging messages unless there is an error
set_log_level("ERROR")

df = pd.read_csv('2330_stock_data.csv', parse_dates=['ds'])
df = add_stock_price_feature(df)
# plt = df.plot(x="ds", y="y", figsize=(15, 5))
df.info()

Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 570 entries, 0 to 569
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Unnamed: 0                570 non-null    int64         
 1   ds                        570 non-null    datetime64[ns]
 2   open_price                570 non-null    float64       
 3   high_price                570 non-null    float64       
 4   low_price                 570 non-null    float64       
 5   y                         570 non-null    float64       
 6   volume                    570 non-null    int64         
 7   foreign                   570 non-null    int64         
 8   investment                570 non-null    int64         
 9   dealer                    570 non-null    int64         
 10  ratio_over_400_shares     570 non-null    float64       
 11  shareholders_400_to_600   570 non-null    int64         
 12  shareholders_600_to_80

In [6]:
from utils import val_mape

param_grid = {
    'ratio_over_400_shares': [0, 5],
    'shareholders_400_to_600': [0, 5],
    'shareholders_600_to_800': [0, 5],
    'shareholders_800_to_1000': [0, 5],
    'ratio_over_1000_shares': [0, 5],
}

results = []
# Iterate over each combination of hyperparameters
for params in ParameterGrid(param_grid):
    # Initialize the NeuralProphet model with current hyperparameters
    print([params])
    m = NeuralProphet(
        yearly_seasonality=False,
        weekly_seasonality=True,
        n_lags=3,
    )
    m = m.add_country_holidays("TW")
    
    lag_regs = (
        # ('volume', 8),
        # ('high_low_diff', 10),
        ('ratio_over_400_shares', params['ratio_over_400_shares']),
        ('shareholders_400_to_600', params['shareholders_400_to_600']),
        ('shareholders_600_to_800', params['shareholders_600_to_800']),
        ('shareholders_800_to_1000', params['shareholders_800_to_1000']),
        ('ratio_over_1000_shares', params['ratio_over_1000_shares']),
    )
    columes = ['ds', 'y']
    for col, lag in lag_regs:
        if lag > 0:
            m.add_lagged_regressor(col, n_lags=lag)
            columes.append(col)
    
    df_train, df_val = m.split_df(df[columes], valid_p=0.2)
    set_random_seed(0)
    metrics = m.fit(df_train, validation_df=df_val)
    
    # Create a new dataframe reaching 365 into the future for our forecast, n_historic_predictions also shows historic data
    df_future = m.make_future_dataframe(df[columes], n_historic_predictions=True, periods=7)

    # Predict the future
    forecast = m.predict(df_future)
    
    rmse = metrics.iloc[-1]['RMSE_val']
    mape = val_mape(df_val, forecast)
    results.append({**params, 'RMSE': rmse, 'MAPE': mape})

[{'ratio_over_1000_shares': 0, 'ratio_over_400_shares': 0, 'shareholders_400_to_600': 0, 'shareholders_600_to_800': 0, 'shareholders_800_to_1000': 0}]


WARNING - (py.warnings._showwarnmsg) - c:\Users\Nick\AppData\Local\Programs\Python\Python311\Lib\site-packages\neuralprophet\df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)

WARNING - (py.warnings._showwarnmsg) - c:\Users\Nick\AppData\Local\Programs\Python\Python311\Lib\site-packages\neuralprophet\df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)



Training: |          | 0/? [00:00<?, ?it/s]

Finding best initial lr:   0%|          | 0/219 [00:00<?, ?it/s]

WARNING - (py.warnings._showwarnmsg) - c:\Users\Nick\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_lightning\utilities\data.py:79: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 14. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.



...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

WARNING - (py.warnings._showwarnmsg) - c:\Users\Nick\AppData\Local\Programs\Python\Python311\Lib\site-packages\neuralprophet\df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)

WARNING - (py.warnings._showwarnmsg) - c:\Users\Nick\AppData\Local\Programs\Python\Python311\Lib\site-packages\neuralprophet\df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)

WARNING - (py.warnings._showwarnmsg) - c:\Users\Nick\AppData\Local\Programs\Python\Python311\Lib\site-packages\neuralprophet\df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_ds = pd

Predicting: |          | 0/? [00:00<?, ?it/s]

In [10]:
results_df = pd.DataFrame(results)
results_df.to_csv('params.csv')
results_df

,ratio_over_1000_shares,ratio_over_400_shares,shareholders_400_to_600,shareholders_600_to_800,shareholders_800_to_1000,RMSE,MAPE
0,0,0,0,0,0,18.046957,0.013945
1,0,0,0,0,5,18.477510,0.014229
2,0,0,0,5,0,19.073133,0.014981
3,0,0,0,5,5,18.020866,0.014073
4,0,0,5,0,0,18.156639,0.013882
5,0,0,5,0,5,17.897186,0.013730
6,0,0,5,5,0,17.930387,0.013586
7,0,0,5,5,5,17.975050,0.013804
8,0,5,0,0,0,18.340185,0.014199
9,0,5,0,0,5,18.731796,0.014369


In [11]:
results_df.sort_values(by="RMSE").head(10)

,ratio_over_1000_shares,ratio_over_400_shares,shareholders_400_to_600,shareholders_600_to_800,shareholders_800_to_1000,RMSE,MAPE
5,0,0,5,0,5,17.897186,0.013730
6,0,0,5,5,0,17.930387,0.013586
7,0,0,5,5,5,17.975050,0.013804
12,0,5,5,0,0,18.016424,0.013581
3,0,0,0,5,5,18.020866,0.014073
0,0,0,0,0,0,18.046957,0.013945
4,0,0,5,0,0,18.156639,0.013882
10,0,5,0,5,0,18.202316,0.013815
16,5,0,0,0,0,18.249451,0.014198
15,0,5,5,5,5,18.257692,0.014107


In [12]:
results_df.sort_values(by="MAPE").head(10)

,ratio_over_1000_shares,ratio_over_400_shares,shareholders_400_to_600,shareholders_600_to_800,shareholders_800_to_1000,RMSE,MAPE
12,0,5,5,0,0,18.016424,0.013581
6,0,0,5,5,0,17.930387,0.013586
5,0,0,5,0,5,17.897186,0.013730
7,0,0,5,5,5,17.975050,0.013804
10,0,5,0,5,0,18.202316,0.013815
4,0,0,5,0,0,18.156639,0.013882
24,5,5,0,0,0,18.442293,0.013944
0,0,0,0,0,0,18.046957,0.013945
13,0,5,5,0,5,18.553362,0.013999
3,0,0,0,5,5,18.020866,0.014073
